In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm, trange
from sklearn.preprocessing import LabelEncoder
import pickle
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Dataset, Subset
import torch_optimizer as optim
import os
import gc

In [2]:
experiments_path = './experiments/amex_transformer_pretrain_v4/'
if not os.path.exists(experiments_path):
    os.makedirs(experiments_path)

In [3]:
X = np.load("./data/X2.npy")
y = np.load("./data/y.npy")
customer_list = np.load("./data/customer_list.npy")
X_fe = np.load("./data/X_fe.npy")

In [5]:
X.shape, y.shape, X_fe.shape

((1383534, 13, 188), (1383534,), (1383534, 1095))

In [6]:
features = pd.read_csv("./data/train_data.csv",index_col=0,nrows=1).columns[2:]
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
dense_features = [col for col in features if col not in cat_features]
features = cat_features + dense_features
features_group = {}
for i, col in enumerate(features):
    g = col[0]
    if g not in features_group:
        features_group[g] = {}
        if col in cat_features:
            features_group[g]['cat'] = [i]
        else:
            features_group[g]['dense'] = [i]
    else:
        if col in cat_features:
            if 'cat' in features_group[g]:
                features_group[g]['cat'].append(i)
            else:
                features_group[g]['cat'] = [i]
        else:
            if 'dense' in features_group[g]:
                features_group[g]['dense'].append(i)
            else:
                features_group[g]['dense'] = [i] 

In [7]:
for a in features_group:
    for b in features_group[a]:
        print(a,b,len(features_group[a][b]))

B cat 2
B dense 38
D cat 9
D dense 87
P dense 3
R dense 28
S dense 21


In [8]:
np.nanmax(X[...,:11].reshape(-1,11),0)+1

array([4., 8., 3., 3., 8., 3., 4., 6., 5., 3., 8.], dtype=float32)

In [9]:
BATCH_SIZE = 1024
EPOCHS = 120
device = torch.device('cuda:1')
dataset = TensorDataset(torch.Tensor(X), torch.Tensor(X_fe))

In [10]:
param = {'d_model': 768,
         'emb_dim': 4,
         'n_layers': 6,
         'n_heads': 4,
         'activation': 'relu',
         'transformer_act': 'relu',
         'use_cls': False,
         'input_norm': False,
         'input_layers': 0,
         'tanh_scale': 0.3450840441113073,
         'input_dropout': 0.10803114983077852,
         'hidden_dropout': 0.23771720623629086,
         'final_dropout': 0.1803505437404746,
         'transformer_dropout': 0.24608225028381883,
         'output_layers': 'mlp2',
         'pe_std': 0.7232348735328199,
         'optimizer': 'Lamb',
         'lr': 0.01,
         'weight_decay': 0.051712665649902206,
         'optimizer_alpha': 0.07931217763278503,
         'optimizer_beta': 0.0070573347096303885}

In [11]:
class TanhEstimator(nn.Module):
    def __init__(self, inp_size, tanh_scale=0.1):
        super().__init__()
        self.alpha = nn.Parameter(tanh_scale * torch.ones(inp_size))
        self.beta = nn.Parameter(torch.zeros(inp_size))
    def forward(self, inp):
        x = torch.tanh(self.alpha * inp + self.beta)
        return x

class AmexModel(nn.Module):
    def __init__(self, params):
        super().__init__()
        d_model = params['d_model']
        emb_dim = params['emb_dim']
        n_layers = params['n_layers']
        n_heads = 2**params['n_heads']
        
        if params['activation'] == 'relu':
            activation = nn.ReLU()
        elif params['activation'] == 'gelu':
            activation = nn.GELU()
        elif params['activation'] == 'mish':
            activation = nn.Mish()
        
        self.use_cls = params['use_cls']
        
        self.n_cat = [4, 8, 3, 3, 8, 3, 4, 6, 5, 3, 8]
        self.n_dense = 177
        self.features_group = features_group
        self.inp_emb = nn.ModuleDict()
        for key1 in self.features_group:
            self.inp_emb[key1] = nn.ModuleDict()
            for key2 in self.features_group[key1]:
                if key2 == 'cat':
                    self.inp_emb[key1][key2] = nn.ModuleList()
                    for i in self.features_group[key1][key2]:
                        self.inp_emb[key1][key2].append(nn.Embedding(self.n_cat[i],emb_dim))
                else:
                    d = len(self.features_group[key1][key2])
                    if 'cat' in self.features_group[key1]:
                        d += len(self.features_group[key1]['cat']) * emb_dim
                    self.inp_emb[key1][key2] = nn.Sequential(nn.Linear(d, d_model),
                                                             nn.Dropout(0.1),
                                                             nn.Mish(),
                                                             nn.Linear(d_model,d_model))
        
        if params['input_norm']:
            self.norm = nn.BatchNorm1d(self.n_dense)
        else:
            self.norm = nn.Identity()
            
        self.dense_norm = TanhEstimator(self.n_dense, params['tanh_scale'])
        self.post_norm = nn.BatchNorm1d(len(self.features_group) * d_model)
        self.proj = [nn.Dropout(params['input_dropout']),nn.Linear(len(self.features_group) * d_model, d_model)]
        for _ in range(params['input_layers']):
            self.proj.extend([nn.Dropout(params['hidden_dropout']), activation, nn.Linear(d_model, d_model)])
        self.proj = nn.Sequential(*self.proj)
        
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model,n_heads,4*d_model,
                                                                            params['transformer_dropout'],
                                                                            activation=params['transformer_act'],
                                                                            norm_first=True,batch_first=True)
                                                 ,n_layers)
        
        if params['output_layers'] == 'linear':
            self.fc = nn.Sequential(nn.Dropout(params['final_dropout']), nn.Linear(d_model, 1))
        elif params['output_layers'] == 'mlp':
            self.fc = nn.Sequential(nn.Dropout(params['hidden_dropout']), nn.Linear(d_model, d_model), activation,
                                    nn.Dropout(params['final_dropout']), nn.Linear(d_model, 1))
        elif params['output_layers'] == 'mlp2':
            self.fc = nn.Sequential(nn.Dropout(params['hidden_dropout']), nn.Linear(d_model, 4*d_model), activation,
                                    nn.Dropout(params['final_dropout']), nn.Linear(4*d_model, 1))
        
        self.ae_fc = nn.Sequential(nn.Dropout(0.2), nn.Linear(d_model, 1095))
        
        self.pe = nn.Parameter(torch.empty([13,d_model]))
        nn.init.normal_(self.pe, std=params['pe_std'])
        
        self.cls = nn.Parameter(torch.empty(d_model))
        nn.init.normal_(self.cls)
        
            
    def forward(self, inp):
        missing_nodes_mask = torch.all(torch.isnan(inp),dim=-1)
        inp[torch.isnan(inp)] = 0
        
        inp_cat = inp[...,:len(self.n_cat)]
        inp_dense = inp[...,len(self.n_cat):]
#         inp_dense = self.dense_norm(inp_dense)
#         inp_dense = self.norm(inp_dense.transpose(1,2)).transpose(1,2)
        inp = torch.cat([inp_cat,inp_dense],dim=-1)
        X = []
        for key1 in self.features_group:
            if 'cat' in self.features_group[key1]:
                X_list = [inp[...,self.features_group[key1]['dense']]]
                for i, idx in enumerate(self.features_group[key1]['cat']):
                    X_list.append(self.inp_emb[key1]['cat'][i](inp[...,idx].long()))
                X_list = torch.cat(X_list,dim=-1)
                X.append(self.inp_emb[key1]['dense'](X_list))
            else:
                X.append(self.inp_emb[key1]['dense'](inp[...,self.features_group[key1]['dense']]))
        X = torch.cat(X,dim=-1)
        X = X.permute(0,2,1)
        X = self.post_norm(X)
        X = X.permute(0,2,1)
        X = self.proj(X)
        X = X + self.pe
        
        if self.use_cls:
            X = torch.cat([X,self.cls.reshape(1,1,-1).repeat(len(X),1,1)],dim=1)
            mask = torch.cat([missing_nodes_mask,torch.zeros([len(X),1],device=X.device).bool()],dim=1)
            X = self.transformer(X, src_key_padding_mask=mask)
        else:
            X = self.transformer(X, src_key_padding_mask=missing_nodes_mask)
        
        X = X[:,-1]
        
        y = self.ae_fc(X)
        return y

In [12]:
criterion = nn.HuberLoss(delta=3.0)

def train_one_epoch(model, optimizer, scheduler, train_dataloader, device = torch.device('cpu')):
    model.train()
    MA_loss = 0
    count = 0
    for X,y in tqdm(train_dataloader, leave=False):
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        
        pred = model(X)
        mask = ~torch.isnan(y)
        loss = criterion(pred[mask],y[mask])
        loss.backward()
        optimizer.step()
        scheduler.step()
        MA_loss += loss.item() * mask.float().sum()
        count += mask.float().sum()
    MA_loss /= count
    return MA_loss

In [13]:
model_path = experiments_path + f"model.pt"
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=0,shuffle=True,drop_last=True)
model = AmexModel(param).to(device)
if param['optimizer'] == 'Lamb':
    optimizer = optim.Lamb(model.parameters(),
                           lr=param['lr'],
                           weight_decay=param['weight_decay'],
                           betas = (1 - param['optimizer_alpha'], 1 - param['optimizer_beta']))
elif param['optimizer'] == 'AdamW':
    optimizer = torch.optim.AdamW(model.parameters(),
                                   lr=param['lr'],
                                   weight_decay=param['weight_decay'],
                                   betas = (1 - param['optimizer_alpha'], 1 - param['optimizer_beta']))
elif param['optimizer'] == 'Ranger':
    optimizer = optim.Ranger(model.parameters(),
                               lr=param['lr'],
                               weight_decay=param['weight_decay'],
                               betas = (1 - param['optimizer_alpha'], 1 - param['optimizer_beta']))
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, EPOCHS*len(train_loader))

best_score = -1.0
for epoch in trange(EPOCHS):
    train_loss = train_one_epoch(model, optimizer, scheduler, train_loader, device)
    print(f"epoch {epoch}")
    print(f"train_loss {train_loss}")

  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/1351 [00:00<?, ?it/s]

epoch 0
train_loss 0.1177452951669693


  0%|          | 0/1351 [00:00<?, ?it/s]

epoch 1
train_loss 0.07917426526546478


...

epoch 117
train_loss 0.051378071308135986


  0%|          | 0/1351 [00:00<?, ?it/s]

epoch 118
train_loss 0.05140624940395355


  0%|          | 0/1351 [00:00<?, ?it/s]

epoch 119
train_loss 0.05143903195858002


In [14]:
torch.save(model.state_dict(), model_path)